In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
!pip install dicomsdl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.5 MB/s eta 0:00:00a 0:00:010m


# Converting images to png

In [4]:
import dicomsdl as dicoml
import cv2
import pydicom

from joblib import Parallel, delayed
import glob
import time
import numpy as np
import os
from matplotlib import pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras

ModuleNotFoundError: No module named 'dicomsdl'

In [4]:
# image_dir_dicomsdl = '/kaggle/working/png_file_dic/'
# os.makedirs(image_dir_dicomsdl, exist_ok=True)
# IMAGES_TO_PROCESS = 500
# train_images = glob.glob("/kaggle/input/rsna-breast-cancer-detection/train_images/*/*.dcm")

# def process(f, size=512, save_folder=None, extension="png"):    
#     patient = f.split('/')[-2]
#     image_name = f.split('/')[-1][:-4]
#     dicom = dicoml.open(f)
#     img = dicom.pixelData()
#     img = (img - img.min()) / (img.max() - img.min())
#     if dicom.getPixelDataInfo()['PhotometricInterpretation'] == "MONOCHROME1":
#         img = 1 - img
#     image = (img * 255).astype(np.uint8)
#     img = cv2.resize(image, (size, size))
#     file_name = f'{save_folder}' + f"{patient}_{image_name}.{extension}"
#     cv2.imwrite(file_name, img)

In [5]:
# %%time
# Parallel(n_jobs=4)(
#     delayed(process)(f, size = 512, save_folder = image_dir_dicomsdl)
#     for f in train_images[:IMAGES_TO_PROCESS]
# )

CPU times: user 4.53 s, sys: 141 ms, total: 4.67 s
Wall time: 3min 56s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

# Preprocessing

In [4]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [68]:
df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
df.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True


In [69]:
df['filepath'] = 'kaggle/working/png_file_dic/' + df['patient_id'].astype(str) + '_' + df['image_id'].astype(str)
df.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,filepath
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False,kaggle/working/png_file_dic/10006_462822612
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False,kaggle/working/png_file_dic/10006_1459541791
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False,kaggle/working/png_file_dic/10006_1864590858
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False,kaggle/working/png_file_dic/10006_1874946579
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True,kaggle/working/png_file_dic/10011_220375232


In [64]:
data = tf.data.Dataset.from_tensor_slices((df["image_id"], df["patient_id"], df["cancer"]))

In [65]:
data.take(1)

<TakeDataset shapes: ((), (), ()), types: (tf.int64, tf.int64, tf.int64)>

In [66]:
data

<TensorSliceDataset shapes: ((), (), ()), types: (tf.int64, tf.int64, tf.int64)>

In [67]:
def extract_image(image_id, patient_id, cancer):
    path = 'kaggle/working/png_file_dic/' + str(patient_id.numpy()) + '_' + str(image_id.numpy())
    print(path)
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image)
    return image, cancer

data = data.map(extract_image)

AttributeError: in user code:

    /tmp/ipykernel_27/970488665.py:2 extract_image  *
        path = 'kaggle/working/png_file_dic/' + str(patient_id.numpy()) + '_' + str(image_id.numpy())
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/ops.py:401 __getattr__
        self.__getattribute__(name)

    AttributeError: 'Tensor' object has no attribute 'numpy'


In [31]:
data = data.batch(500)

In [32]:
def preprocess(image, cancer):
    image = tf.image.resize(image, (512, 512))
    image = image / 255.0 # normalize to [0,1] range
    return image, cancer

data = data.map(preprocess)

In [33]:
X, y = data.unbatch()

2023-01-23 16:05:58.233009: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at whole_file_read_ops.cc:116 : Not found: kaggle/working/png_file_dic/Tensor("args_1:0", shape=(1,), dtype=int64)_Tensor("args_0:0", shape=(1,), dtype=int64); No such file or directory
2023-01-23 16:05:58.244913: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at whole_file_read_ops.cc:116 : Not found: kaggle/working/png_file_dic/Tensor("args_1:0", shape=(1,), dtype=int64)_Tensor("args_0:0", shape=(1,), dtype=int64); No such file or directory
2023-01-23 16:05:58.245000: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at whole_file_read_ops.cc:116 : Not found: kaggle/working/png_file_dic/Tensor("args_1:0", shape=(1,), dtype=int64)_Tensor("args_0:0", shape=(1,), dtype=int64); No such file or directory
2023-01-23 16:05:58.245059: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at whole_file_read_ops.cc:116 : Not found: kaggle/working/png_file

NotFoundError: kaggle/working/png_file_dic/Tensor("args_1:0", shape=(1,), dtype=int64)_Tensor("args_0:0", shape=(1,), dtype=int64); No such file or directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext]